# NGS pipeline for Alzheimer's Disease RNA-seq data

For this project I have used only one control and one sample of someone of advanced AD.

In [1]:
import os

Set up some global variables:

In [2]:
# Dir with the bash files
os.environ['DIR'] = '/home/mees/NGS_Alzheimer/'
# Working directory to the sequencing data (I use an external disk, since I dont have a lot of space left on the pc)
os.environ['WORKDIR'] = '/media/mees/Elements/ngs_data/'
# The directory to the downloaded genome
os.environ['GENOMEDIR'] = '/media/mees/Elements/ngs_data/genomes/'
# Location to save the indexed reference genome
os.environ['REF_GENOME'] = '/media/mees/Elements/ngs_data/genomes/ref_genome/'

First download the human genome for reference and save it as genome.fa in $GENOMEDIR

In [ ]:
!wget ftp://ftp.ensembl.org/pub/release-100/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz
!gzip -d Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz
!mv Homo_sapiens.GRCh38.dna.primary_assembly.fa $GENOMEDIR/genome.fa

In [11]:
!mkdir -p $WORKDIR
!bash download_data.sh -d $WORKDIR


2020-06-23T21:46:26 prefetch.2.10.7: 1) 'SRR837437' is found locally

2020-06-23T21:46:26 prefetch.2.10.7: 2) 'SRR837471' is found locally


join   :|  0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.7 0.7 0.7 0.7 0.7 0.7 0.7 0.7 0.7 0.7 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.99-  1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.1 1.1 1.1 1.1 1.1 1.1 1.1 1.1 1.1 1.1 1.2 1.2 1.2 1.2 1.2 1.2 1.2 1.2 1.2 1.2 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.4 1.4 1.4 1.4 1.4 1.4 1.4 1.4 1.4 1.4 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.6 1.6 1.6 1.6 1.6 1.6 1.6 1.6 1.6 1.6 1.7 1.7 1.7 1.7 1.7 1.7 1.7 1.7 1.7 1.7 1.8 1.8 1.8 1.8 1.8 1.8 1.8 1.8 1.8 1.8 1.9 1.9 1.9 1.9 1.9 1.9 1.9 1.9 1.9 1.9 2.0 2.0 2.0 2.0 2.0 2.0 2.0 2.0 2.0 2.0 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.2 2.2 2.2 2.2 2.2 2.2 2.2 2.2 2.2 2.2 2.3 2.3 2.3 2.3 2.3 2.3 2.3 2.3 2.3 2.3 2.4 2.4 2.4 2.4 2.4 2.4 2.

86.386.486.486.486.486.486.486.486.486.486.486.586.586.586.586.586.586.586.586.586.586.686.686.686.686.686.686.686.686.686.686.786.786.786.786.786.786.786.786.786.786.886.886.886.886.886.886.886.886.886.886.986.986.986.986.986.986.986.986.986.99- 87.087.087.087.087.087.087.087.087.087.087.187.187.187.187.187.187.187.187.187.187.287.287.287.287.287.287.287.287.287.287.387.387.387.387.387.387.387.387.387.387.487.487.487.487.487.487.487.487.487.487.587.587.587.587.587.587.587.587.587.587.687.687.687.687.687.687.687.687.687.687.787.787.787.787.787.787.787.787.787.787.887.887.887.887.887.887.887.887.887.887.987.987.987.987.987.987.987.987.987.988.088.088.088.088.088.088.088.088.088.088.188.188.188.188.188.188.188.188.188.188.288.288.288.288.288.288.288.288.288.288.388.388.388.388.388.388.388.388.388.388.488.488.488.488.488.488.488.488.488.488.588.588.588.588.588.588.588.588.588.588.688.688.688.688.688.688.688.688.688.688.788.788.788.788.788.788.788.788.788.788.888.888.888.888.888.888.888.88

concat :|  0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.7 0.7 0.7 0.7 0.7 0.7 0.7 0.7 0.7 0.7 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.99-  1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.1 1.1 1.1 1.1 1.1 1.1 1.1 1.1 1.1 1.1 1.2 1.2 1.2 1.2 1.2 1.2 1.2 1.2 1.2 1.2 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.4 1.4 1.4 1.4 1.4 1.4 1.4 1.4 1.4 1.4 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.6 1.6 1.6 1.6 1.6 1.6 1.6 1.6 1.6 1.6 1.7 1.7 1.7 1.7 1.7 1.7 1.7 1.7 1.7 1.7 1.8 1.8 1.8 1.8 1.8 1.8 1.8 1.8 1.8 1.8 1.9 1.9 1.9 1.9 1.9 1.9 1.9 1.9 1.9 1.9 2.0 2.0 2.0 2.0 2.0 2.0 2.0 2.0 2.0 2.0 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.2 2.2 2.2 2.2 2.2 2.2 2.2 2.2 2.2 2.2 2.3 2.3 2.3 2.3 2.3 2.3 2.3 2.3 2.3 2.3 2.4 2.4 2.4 2.4 2.4 2.4 2.

81.881.881.981.981.981.981.981.981.981.981.981.982.082.082.082.082.082.082.082.082.082.082.182.182.182.182.182.182.182.182.182.182.282.282.282.282.282.282.282.282.282.282.382.382.382.382.382.382.382.382.382.382.482.482.482.482.482.482.482.482.482.482.582.582.582.582.582.582.582.582.582.582.682.682.682.682.682.682.682.682.682.682.782.782.782.782.782.782.782.782.782.782.882.882.882.882.882.882.882.882.882.882.982.982.982.982.982.982.982.982.982.99- 83.083.083.083.083.083.083.083.083.083.083.183.183.183.183.183.183.183.183.183.183.283.283.283.283.283.283.283.283.283.283.383.383.383.383.383.383.383.383.383.383.483.483.483.483.483.483.483.483.483.483.583.583.583.583.583.583.583.583.583.583.683.683.683.683.683.683.683.683.683.683.783.783.783.783.783.783.783.783.783.783.883.883.883.883.883.883.883.883.883.883.983.983.983.983.983.983.983.983.983.984.084.084.084.084.084.084.084.084.084.084.184.184.184.184.184.184.184.184.184.184.284.284.284.284.284.284.284.284.284.284.384.384.384.384.384.384.38

join   :|  0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.7 0.7 0.7 0.7 0.7 0.7 0.7 0.7 0.7 0.7 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.99-  1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.1 1.1 1.1 1.1 1.1 1.1 1.1 1.1 1.1 1.1 1.2 1.2 1.2 1.2 1.2 1.2 1.2 1.2 1.2 1.2 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.4 1.4 1.4 1.4 1.4 1.4 1.4 1.4 1.4 1.4 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.6 1.6 1.6 1.6 1.6 1.6 1.6 1.6 1.6 1.6 1.7 1.7 1.7 1.7 1.7 1.7 1.7 1.7 1.7 1.7 1.8 1.8 1.8 1.8 1.8 1.8 1.8 1.8 1.8 1.8 1.9 1.9 1.9 1.9 1.9 1.9 1.9 1.9 1.9 1.9 2.0 2.0 2.0 2.0 2.0 2.0 2.0 2.0 2.0 2.0 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.2 2.2 2.2 2.2 2.2 2.2 2.2 2.2 2.2 2.2 2.3 2.3 2.3 2.3 2.3 2.3 2.3 2.3 2.3 2.3 2.4 2.4 2.4 2.4 2.4 2.4 2.

83.483.583.583.583.583.583.583.583.583.583.583.683.683.683.683.683.683.683.683.683.683.783.783.783.783.783.783.783.783.783.783.883.883.883.883.883.883.883.883.883.883.983.983.983.983.983.983.983.983.983.984.084.084.084.084.084.084.084.084.084.084.184.184.184.184.184.184.184.184.184.184.284.284.284.284.284.284.284.284.284.284.384.384.384.384.384.384.384.384.384.384.484.484.484.484.484.484.484.484.484.484.584.584.584.584.584.584.584.584.584.584.684.684.684.684.684.684.684.684.684.684.784.784.784.784.784.784.784.784.784.784.884.884.884.884.884.884.884.884.884.884.984.984.984.984.984.984.984.984.984.99- 85.085.085.085.085.085.085.085.085.085.085.185.185.185.185.185.185.185.185.185.185.285.285.285.285.285.285.285.285.285.285.385.385.385.385.385.385.385.385.385.385.485.485.485.485.485.485.485.485.485.485.585.585.585.585.585.585.585.585.585.585.685.685.685.685.685.685.685.685.685.685.785.785.785.785.785.785.785.785.785.785.885.885.885.885.885.885.885.885.885.885.985.985.985.985.985.985.985.98

concat :|  0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.7 0.7 0.7 0.7 0.7 0.7 0.7 0.7 0.7 0.7 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.99-  1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.1 1.1 1.1 1.1 1.1 1.1 1.1 1.1 1.1 1.1 1.2 1.2 1.2 1.2 1.2 1.2 1.2 1.2 1.2 1.2 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.4 1.4 1.4 1.4 1.4 1.4 1.4 1.4 1.4 1.4 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.6 1.6 1.6 1.6 1.6 1.6 1.6 1.6 1.6 1.6 1.7 1.7 1.7 1.7 1.7 1.7 1.7 1.7 1.7 1.7 1.8 1.8 1.8 1.8 1.8 1.8 1.8 1.8 1.8 1.8 1.9 1.9 1.9 1.9 1.9 1.9 1.9 1.9 1.9 1.9 2.0 2.0 2.0 2.0 2.0 2.0 2.0 2.0 2.0 2.0 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.2 2.2 2.2 2.2 2.2 2.2 2.2 2.2 2.2 2.2 2.3 2.3 2.3 2.3 2.3 2.3 2.3 2.3 2.3 2.3 2.4 2.4 2.4 2.4 2.4 2.4 2.

81.981.981.981.981.982.082.082.082.082.082.082.082.082.082.082.182.182.182.182.182.182.182.182.182.182.282.282.282.282.282.282.282.282.282.282.382.382.382.382.382.382.382.382.382.382.482.482.482.482.482.482.482.482.482.482.582.582.582.582.582.582.582.582.582.582.682.682.682.682.682.682.682.682.682.682.782.782.782.782.782.782.782.782.782.782.882.882.882.882.882.882.882.882.882.882.982.982.982.982.982.982.982.982.982.99- 83.083.083.083.083.083.083.083.083.083.083.183.183.183.183.183.183.183.183.183.183.283.283.283.283.283.283.283.283.283.283.383.383.383.383.383.383.383.383.383.383.483.483.483.483.483.483.483.483.483.483.583.583.583.583.583.583.583.583.583.583.683.683.683.683.683.683.683.683.683.683.783.783.783.783.783.783.783.783.783.783.883.883.883.883.883.883.883.883.883.883.983.983.983.983.983.983.983.983.983.984.084.084.084.084.084.084.084.084.084.084.184.184.184.184.184.184.184.184.184.184.284.284.284.284.284.284.284.284.284.284.384.384.384.384.384.384.384.384.384.384.484.484.484.48

After downloading the data and convert the .sra files to .fastq files, use FastQC to determine read quality.
In the article, they alreade mentioned they had to trim the 3' adapter.

In [13]:
!./fastqc.sh -d $WORKDIR 

Started analysis of SRR837437.sra.fastq
Approx 5% complete for SRR837437.sra.fastq
Approx 10% complete for SRR837437.sra.fastq
Approx 15% complete for SRR837437.sra.fastq
Approx 20% complete for SRR837437.sra.fastq
Approx 25% complete for SRR837437.sra.fastq
Approx 30% complete for SRR837437.sra.fastq
Approx 35% complete for SRR837437.sra.fastq
Approx 40% complete for SRR837437.sra.fastq
Approx 45% complete for SRR837437.sra.fastq
Approx 50% complete for SRR837437.sra.fastq
Approx 55% complete for SRR837437.sra.fastq
Approx 60% complete for SRR837437.sra.fastq
Approx 65% complete for SRR837437.sra.fastq
Approx 70% complete for SRR837437.sra.fastq
Approx 75% complete for SRR837437.sra.fastq
Approx 80% complete for SRR837437.sra.fastq
Approx 85% complete for SRR837437.sra.fastq
Approx 90% complete for SRR837437.sra.fastq
Approx 95% complete for SRR837437.sra.fastq
Analysis complete for SRR837437.sra.fastq
Started analysis of SRR837471.sra.fastq
Approx 5% complete for SRR837471.sra.fastq


I like to open the created reports of fastq to see if there are some abnormalities.

In [7]:
!firefox $WORKDIR/*.html

From these reports, we can see that the Illumina small adapter 3' is over represented. Thus we have to remove this.

In the article, we find that they used Illumina's TruSeq Small RNA sample kit. And from Illumina's website, we retrieved this information:

TruSeq Small RNA:

    TGGAATTCTCGGGTGCCAAGG
    
Therefore, we are removing the adapters with cutadapt

In [12]:
!./trim.sh -d $WORKDIR

This is cutadapt 1.15 with Python 3.6.9
Command line parameters: -a TGGAATTCTCGGGTGCCAAGG -a TCCTGTACTGAGCTGCCCCG -m 18 -o trimmed/SRR837437.sra.fastq SRR837437.sra.fastq
Running on 1 core
Trimming 2 adapters with at most 10.0% errors in single-end mode ...
Finished in 421.40 s (24 us/read; 2.54 M reads/minute).

=== Summary ===

Total reads processed:              17,830,678
Reads with adapters:                17,824,962 (100.0%)
Reads that were too short:             787,240 (4.4%)
Reads written (passing filters):    17,043,438 (95.6%)

Total basepairs processed:   891,533,900 bp
Total written (filtered):    371,124,932 bp (41.6%)

=== Adapter 1 ===

Sequence: TGGAATTCTCGGGTGCCAAGG; Type: regular 3'; Length: 21; Trimmed: 17203587 times.

No. of allowed errors:
0-9 bp: 0; 10-19 bp: 1; 20-21 bp: 2

Bases preceding removed adapters:
  A: 47.4%
  C: 1.0%
  G: 45.3%
  T: 6.2%
  none/other: 0.1%

Overview of removed sequences
length	count	expect	max.err	error counts
3	14	278604.3	0	14
4	7	

FastQC analysis after trimming the adapters.

In [ ]:
!./fastqc.sh -d $WORKDIR/trimmed/

Started analysis of SRR837437.sra.fastq
Approx 5% complete for SRR837437.sra.fastq
Approx 10% complete for SRR837437.sra.fastq
Approx 15% complete for SRR837437.sra.fastq
Approx 20% complete for SRR837437.sra.fastq
Approx 25% complete for SRR837437.sra.fastq
Approx 30% complete for SRR837437.sra.fastq
Approx 35% complete for SRR837437.sra.fastq
Approx 40% complete for SRR837437.sra.fastq
Approx 45% complete for SRR837437.sra.fastq
Approx 50% complete for SRR837437.sra.fastq
Approx 55% complete for SRR837437.sra.fastq
Approx 60% complete for SRR837437.sra.fastq
Approx 65% complete for SRR837437.sra.fastq
Approx 70% complete for SRR837437.sra.fastq
Approx 75% complete for SRR837437.sra.fastq
Approx 80% complete for SRR837437.sra.fastq
Approx 85% complete for SRR837437.sra.fastq
Approx 90% complete for SRR837437.sra.fastq
Approx 95% complete for SRR837437.sra.fastq
Analysis complete for SRR837437.sra.fastq
Started analysis of SRR837471.sra.fastq
Approx 5% complete for SRR837471.sra.fastq


In [ ]:
!firefox $WORKDIR/trimmed/*.html

Now we still have over represented sequences --> see fastqc html reports
We gonna remove these also using cutadapt.

In [7]:
!./remove_overrepresented.sh -d $WORKDIR/trimmed/

This is cutadapt 1.15 with Python 3.6.9
Command line parameters: -a TCCTGTACTGAGCTGCCCCG -m 18 -o rm_over_SRR837437.sra.fastq SRR837437.sra.fastq
Running on 1 core
Trimming 1 adapter with at most 10.0% errors in single-end mode ...
Finished in 170.56 s (10 us/read; 6.21 M reads/minute).

=== Summary ===

Total reads processed:              17,664,338
Reads with adapters:                16,697,577 (94.5%)
Reads that were too short:          16,696,055 (94.5%)
Reads written (passing filters):       968,283 (5.5%)

Total basepairs processed:   393,138,178 bp
Total written (filtered):     21,598,495 bp (5.5%)

=== Adapter 1 ===

Sequence: TCCTGTACTGAGCTGCCCCG; Type: regular 3'; Length: 20; Trimmed: 16697577 times.

No. of allowed errors:
0-9 bp: 0; 10-19 bp: 1; 20 bp: 2

Bases preceding removed adapters:
  A: 0.2%
  C: 0.3%
  G: 0.0%
  T: 0.5%
  none/other: 99.1%

Overview of removed sequences
length	count	expect	max.err	error counts
3	1453	276005.3	0	1453
4	434	69001.3	0	434
5	205	17250.3

Now that we have improved the read qualities, we can align the reads to the human reference genome. For this alignment, we use Hisat2.

First thing is to build the reference genome using hisat2-build.
Then we perform the alignment using hisat2.

In [11]:
!./build.sh -g $GENOMEDIR/genome.fa -o $REF_GENOME

Settings:
  Output files: "/media/mees/Elements/ngs_data/genomes/ref_genome//ref_genome.*.ht2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Local offset rate: 3 (one in 8)
  Local fTable chars: 6
  Local sequence length: 57344
  Local sequence overlap between two consecutive indexes: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  /media/mees/Elements/ngs_data/genomes//genome.fa
Reading reference sizes
  Time reading reference sizes: 00:01:30
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:01:29
  Time to read SNPs and splice sites: 00:00:00
Using parameters --bmax 138086675 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing w

  Sorting block time: 00:00:15
Returning block of 30670784 for bucket 11
  bucket 14: 10%
Getting block 15 of 32
  Reserving size (138086675) for bucket 15
  Calculating Z arrays for bucket 15
  Entering block accumulator loop for bucket 15:
  bucket 13: 100%
  Sorting block of length 62107305 for bucket 13
  (Using difference cover)
  bucket 14: 20%
  bucket 15: 10%
  bucket 14: 30%
  bucket 15: 20%
  bucket 14: 40%
  bucket 15: 30%
  bucket 14: 50%
  bucket 15: 40%
  bucket 15: 50%
  bucket 14: 60%
  bucket 15: 60%
  bucket 14: 70%
  Sorting block time: 00:00:34
Returning block of 62107306 for bucket 13
  bucket 15: 70%
  bucket 14: 80%
Getting block 16 of 32
  Reserving size (138086675) for bucket 16
  Calculating Z arrays for bucket 16
  Entering block accumulator loop for bucket 16:
  bucket 15: 80%
  bucket 14: 90%
  bucket 16: 10%
  bucket 15: 90%
  bucket 16: 20%
  bucket 14: 100%
  Sorting block of length 139760453 for bucket 14
  (Using difference cover)
  bucket 15: 100%
  S

  bucket 31: 90%
  bucket 31: 100%
  Sorting block of length 116855486 for bucket 31
  (Using difference cover)
  Sorting block time: 00:00:32
Returning block of 47458124 for bucket 30
Getting block 32 of 32
  Reserving size (138086675) for bucket 32
  Calculating Z arrays for bucket 32
  Entering block accumulator loop for bucket 32:
  bucket 32: 10%
  bucket 32: 20%
  bucket 32: 30%
  bucket 32: 40%
  bucket 32: 50%
  bucket 32: 60%
  bucket 32: 70%
  bucket 32: 80%
  Sorting block time: 00:01:08
Returning block of 113546999 for bucket 28
  bucket 32: 90%
  bucket 32: 100%
  Sorting block of length 45807214 for bucket 32
  (Using difference cover)
  Sorting block time: 00:01:05
Returning block of 114596170 for bucket 29
  Sorting block time: 00:01:08
Returning block of 116855487 for bucket 31
  Sorting block time: 00:00:22
Returning block of 45807215 for bucket 32
Exited GFM loop
fchr[A]: 0
fchr[C]: 869653843
fchr[G]: 1470243264
fchr[T]: 2073417374
fchr[$]: 2945849067
Exiting GFM::bu

Now we can align our data to the indexed human genome

In [ ]:
!./align.sh -g $REF_GENOME -f $WORKDIR/trimmed/